In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import os
os.getcwd()

'C:\\Users\\ajaya'

In [5]:
df1 = pd.read_csv(r"C:\Users\ajaya\Desktop\NUMPYNINJA\HACKATHON\HACKATHON DETAILS\schema_1_ontario_final.csv")
print("Shape:",df1.shape)
df1.columns

Shape: (263639, 12)


Index(['week', 'fsa', 'probable', 'vulnerable', 'is_most_recent',
       'fever_chills_shakes', 'cough', 'shortness_of_breath', 'over_60',
       'any_medical_conditions', 'travel_outside_canada',
       'contact_with_illness'],
      dtype='object')

In [6]:
print("\n First 5 rows:")
df1.head()


 First 5 rows:


,week,fsa,probable,vulnerable,is_most_recent,fever_chills_shakes,cough,shortness_of_breath,over_60,any_medical_conditions,travel_outside_canada,contact_with_illness
0,13,L4C,n,n,y,n,n,n,n,n,n,n
1,13,P1H,y,y,y,n,n,n,y,y,n,y
2,14,N2J,n,y,y,n,n,n,y,n,n,n
3,13,N1G,n,n,y,n,n,n,n,n,n,n
4,13,M1P,n,y,n,n,n,n,n,y,n,n


In [7]:
#INITIAL INSPECTION
print("DATA INSPECTION")
print("\nColumn info:")
df1.info()
print("\nMissing values:")
df1.isnull().sum()
print("\nDuplicates:", df1.duplicated().sum())
print("\nUnique values per column:")
for col in df1.columns:
    print(f"  {col}: {df1[col].nunique()}")

DATA INSPECTION

Column info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263639 entries, 0 to 263638
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   week                    263639 non-null  int64 
 1   fsa                     263639 non-null  object
 2   probable                263639 non-null  object
 3   vulnerable              263639 non-null  object
 4   is_most_recent          263639 non-null  object
 5   fever_chills_shakes     263639 non-null  object
 6   cough                   263639 non-null  object
 7   shortness_of_breath     263639 non-null  object
 8   over_60                 263639 non-null  object
 9   any_medical_conditions  263639 non-null  object
 10  travel_outside_canada   263639 non-null  object
 11  contact_with_illness    263639 non-null  object
dtypes: int64(1), object(11)
memory usage: 24.1+ MB

Missing values:

Duplicates: 241441

Unique values per column:
 

In [8]:
#CLEAN BINARY COLUMNS (y/n -> True/False)
binary_cols = [
    'probable', 'vulnerable', 'is_most_recent', 'fever_chills_shakes', 
    'cough', 'shortness_of_breath', 'over_60', 'any_medical_conditions', 
    'travel_outside_canada', 'contact_with_illness'
]

for col in binary_cols:
    df1[col] = df1[col].map({'y': True, 'n': False}).astype(bool)

print("Binary columns converted")
df1[binary_cols].head()

Binary columns converted


,probable,vulnerable,is_most_recent,fever_chills_shakes,cough,shortness_of_breath,over_60,any_medical_conditions,travel_outside_canada,contact_with_illness
0,False,False,True,False,False,False,False,False,False,False
1,True,True,True,False,False,False,True,True,False,True
2,False,True,True,False,False,False,True,False,False,False
3,False,False,True,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,True,False,False


In [10]:
#REMOVE DUPLICATES & MISSING VALUES
initial_rows = len(df1)
df1_clean = df1.drop_duplicates()
df1_clean = df1_clean.dropna()

print(f" Cleaning Summary:")
print(f"  Before: {initial_rows:,} rows")
print(f"  After:  {len(df1_clean):,} rows")
print(f"  Removed: {initial_rows - len(df1_clean):,} rows ({(1-len(df1_clean)/initial_rows)*100:.1f}%)")

df1 = df1_clean  # Update main dataframe

 Cleaning Summary:
  Before: 263,639 rows
  After:  22,198 rows
  Removed: 241,441 rows (91.6%)


In [11]:
#FEATURE ENGINEERING
print("🔧 CREATING NEW FEATURES")

# Risk score
risk_cols = ['probable', 'vulnerable', 'over_60', 'any_medical_conditions']
df1['risk_score'] = df1[risk_cols].sum(axis=1)

# Symptom count
symptom_cols = ['fever_chills_shakes', 'cough', 'shortness_of_breath']
df1['symptom_count'] = df1[symptom_cols].sum(axis=1)

# High risk flag
df1['high_risk'] = (df1['risk_score'] >= 2) | (df1['symptom_count'] > 0)

print("New features created")
print(df1[['risk_score', 'symptom_count', 'high_risk']].head())

🔧 CREATING NEW FEATURES
New features created
   risk_score  symptom_count  high_risk
0           0              0      False
1           4              0       True
2           2              0       True
3           0              0      False
4           2              0       True


In [12]:
#SUMMARY STATISTICS
print(" FINAL SUMMARY")
print("="*50)
print(f"Rows: {len(df1):,}")
print(f"Columns: {len(df1.columns)}")
print(f"Weeks: {df1['week'].min()} - {df1['week'].max()}")
print(f"Unique FSAs: {df1['fsa'].nunique():,}")
print(f"High-risk cases: {df1['high_risk'].sum():,} ({df1['high_risk'].mean():.1%})")
print(f"Avg risk score: {df1['risk_score'].mean():.2f}")
print(f"Avg symptoms: {df1['symptom_count'].mean():.2f}")

# Weekly summary table
weekly_summary = df1.groupby('week').agg({
    'fsa': 'count',
    'risk_score': 'mean',
    'high_risk': 'mean',
    'symptom_count': 'mean'
}).round(2)
weekly_summary.columns = ['Cases', 'Avg Risk', '% High Risk', 'Avg Symptoms']
print("\n Weekly Summary:")
weekly_summary

 FINAL SUMMARY
Rows: 22,198
Columns: 15
Weeks: 13 - 15
Unique FSAs: 516
High-risk cases: 17,319 (78.0%)
Avg risk score: 1.66
Avg symptoms: 0.56

 Weekly Summary:


,Cases,Avg Risk,% High Risk,Avg Symptoms
week,,,,
13,11666,1.71,0.80,0.63
14,8871,1.66,0.78,0.54
15,1661,1.28,0.60,0.15


In [13]:
df1

,week,fsa,probable,vulnerable,is_most_recent,fever_chills_shakes,cough,shortness_of_breath,over_60,any_medical_conditions,travel_outside_canada,contact_with_illness,risk_score,symptom_count,high_risk
0,13,L4C,False,False,True,False,False,False,False,False,False,False,0,0,False
1,13,P1H,True,True,True,False,False,False,True,True,False,True,4,0,True
2,14,N2J,False,True,True,False,False,False,True,False,False,False,2,0,True
3,13,N1G,False,False,True,False,False,False,False,False,False,False,0,0,False
4,13,M1P,False,True,False,False,False,False,False,True,False,False,2,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263463,13,K0L,False,False,False,False,False,False,False,False,True,False,0,0,False
263466,14,N7G,True,False,True,False,False,False,False,False,False,True,1,0,False
263479,14,L1X,True,True,True,False,False,True,False,True,False,False,3,1,True
263536,13,M4A,True,False,True,False,True,True,False,False,False,False,1,2,True
